Notebook based on the [oficial example](https://github.com/openai/openai-python/blob/main/examples/finetuning/finetuning-classification.ipynb)

## Step 1: Use smtag to generate your Seq2seq file into GPT3 supported format

In [25]:
!python -m smtag.cli.seq2seq.gpt3_finetune "/data/seq2seq/sd-seq2seq-with-html-tags.csv"

Wrong number of task--input in line. Possible lack of 
                                    delimiter. Skipping line to next one. Tag entities, find causality, and experimental methods:  H, I Blood glucose during  

Wrong number of task--input in line. Possible lack of 
                                    delimiter. Skipping line to next one. Tag entities, find causality, and experimental methods: Figure 5. The anaphase rDNA loop is condensin dependent.

Parameter 'function'=<function Gpt3FineTuner._convert_to_gpt at 0x7f720a38c9d0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed

### Use the data preparation tool of openai

In [26]:
!pip install --upgrade -q openai

In [27]:
!pip install -q wandb

In [28]:
!openai tools fine_tunes.prepare_data -f /data/seq2seq/gpt_ready_train.jsonl -q

Analyzing...

- Your file contains 16355 prompt-completion pairs
- There are 44 duplicated prompt-completion sets. These are rows: [256, 400, 804, 1014, 1434, 1559, 2911, 3290, 4152, 4575, 4611, 5220, 5320, 6337, 6344, 6864, 7226, 7928, 8608, 10120, 10409, 11015, 11102, 11483, 11603, 11605, 11894, 11895, 11896, 11909, 11918, 12792, 12801, 13016, 13154, 13636, 14337, 14573, 14777, 15037, 15051, 15300, 15452, 15676]
- All prompts end with suffix ` \n\n###\n\n `
- All completions end with suffix ` </experiment> [END]`. This suffix seems very long. Consider replacing with a shorter suffix, such as `\n`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 44 duplicate rows [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `/data/seq2seq/gpt_ready_train_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "/data/seq2seq/gpt_ready_train_prepared.jsonl"



In [29]:
!openai tools fine_tunes.prepare_data -f /data/seq2seq/gpt_ready_eval.jsonl -q

Analyzing...

- Your file contains 2069 prompt-completion pairs
- There are 1 duplicated prompt-completion sets. These are rows: [1363]
- All prompts end with suffix ` \n\n###\n\n `
- All prompts start with prefix `Tag entities, find causality, and experimental methods: `. Fine-tuning doesn't require the instruction specifying the task, or a few-shot example scenario. Most of the time you should only add the input data into the prompt, and the desired output into the completion
- All completions end with suffix ` </experiment> [END]`. This suffix seems very long. Consider replacing with a shorter suffix, such as `\n`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 1 duplicate rows [Y/n]: Y
- [Recommended] Remove prefix `Tag entities, find causality, and experimental methods: ` from all prompts [Y/n]: Y
/opt/conda/lib/python3.8/site-packages/openai/validators.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [24]:
!rm -rf /data/seq2seq/*.jsonl

### Fine-tuning



In [ ]:
!openai api fine_tunes.create \
        -t /data/seq2seq/gpt_ready_train_prepared.jsonl \
        -v /data/seq2seq/gpt_ready_eval_prepared.jsonl \
        -m text-davinci-002 \
        --suffix 'sd-gpt-model-' \
        --n_epochs 2 \
        --batch_size 256 \
        --learning_rate_multiplier 0.2 \
        --prompt_loss_weight PROMPT_LOSS_WEIGHT \



In [ ]:
!openai api fine_tunes.results -i ft-2zaA7qi0rxJduWQpdvOvmGn3 > /data/seq2seq/sd-gpt3-model-result.csv

In [ ]:
results = pd.read_csv('/data/seq2seq/sd-gpt3-model-result.csv')
results

### Using the model



We need to use the same separator following the prompt which we used during fine-tuning. In this case it is \n\n###\n\n. Since we're concerned with classification, we want the temperature to be as low as possible, and we only require one token completion to determine the prediction of the model.

In [ ]:
ft_model = 'NAME HERE'
res = openai.Completion.create( model=ft_model, 
                                prompt=test['prompt'][0] + '\n\n###\n\n', 
                                max_tokens=1, 
                                temperature=0,
                                top_p=1,
                                n=1,
                                stream=False,
                                echo=False,
                                frequency_penalt=0,
                                presence_penalty=0,
                                stop=["[END]"],
                                best_of=1,
                                logit_bias=None
                            )
res['choices'][0]['text']

In [ ]:
res = openai.Completion.create(model=ft_model, prompt=test['prompt'][0] + '\n\n###\n\n', max_tokens=1, temperature=0, logprobs=2)
res['choices'][0]['logprobs']['top_logprobs'][0]